# Kaggle Capstone Notebook — Google Cloud (ADK/SDK) Integration

This notebook is a template for a Kaggle Capstone project that integrates with Google Cloud services (GCS, BigQuery, Vertex AI, Secret Manager, Cloud SQL) using the Google Cloud SDK / client libraries ('ADK' shorthand). It is written to run in a Kaggle Notebook environment, where you can upload a **service account JSON** as a Kaggle Dataset (private) and mount it, or use Application Default Credentials locally.

> ⚠️ Security reminder: Never commit service account keys to a public repo. In Kaggle, add the service account JSON as a **private dataset** and attach it to the kernel. Remove keys when done.



## Quick overview

This notebook includes:
- Authentication patterns for Kaggle and local Jupyter
- Optional installs for Google Cloud client libraries and ML tooling
- Examples: GCS upload/download, BigQuery load & query, Vertex AI embedding/LLM pattern (pseudo-code), Cloud SQL connection
- RAG demo using toy data and a simple TF retriever (no external deps)
- Instructions on how to adapt for real Kaggle datasets and Capstone submission



## 0) How to provide Google credentials in Kaggle

1. Create a service account in GCP with minimal scopes (Storage Object Admin for GCS access, BigQuery Data Viewer for read-only, Secret Manager Accessor if needed, Vertex AI User if using Vertex).
2. Download the JSON key and add it to a **private Kaggle dataset** (not the public dataset).
3. In the Kaggle notebook: open the "Add data" panel and attach the private dataset. The key file will appear under `/kaggle/input/<dataset-name>/`.
4. Set environment variable `GOOGLE_APPLICATION_CREDENTIALS` to that path in the notebook before using client libraries.

Example path in Kaggle: `/kaggle/input/my-gcp-key/service-account.json`

## 1) Optional installs

Uncomment to install additional libraries in the Kaggle runtime. Kaggle typically has many packages preinstalled; adjust as needed.

```python
# !pip install --quiet google-cloud-storage google-cloud-bigquery google-cloud-secret-manager google-cloud-aiplatform sqlalchemy psycopg2-binary
# Optional for embeddings locally: sentence-transformers faiss-cpu
```

In [ ]:
# Standard imports
import os
from pathlib import Path
import json
import pandas as pd
import re
import math
print('Imports ready')

## 2) Set project variables and auth

Edit these variables: `PROJECT_ID`, `BUCKET`, `REGION`. For Kaggle, point `SERVICE_ACCOUNT_PATH` to the uploaded private dataset file.


In [ ]:
# Configuration - EDIT these for your GCP project
PROJECT_ID = 'your-gcp-project'
BUCKET = 'your-gcs-bucket'
REGION = 'us-central1'
# In Kaggle, upload your service account JSON as a private dataset and set the path below
SERVICE_ACCOUNT_PATH = '/kaggle/input/my-gcp-key/service-account.json'  # update if different

# Set credentials for Google client libs
if os.path.exists(SERVICE_ACCOUNT_PATH):
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = SERVICE_ACCOUNT_PATH
    print('Using service account:', SERVICE_ACCOUNT_PATH)
else:
    print('Service account JSON not found at', SERVICE_ACCOUNT_PATH)
    print('In local dev, use gcloud auth application-default login or set GOOGLE_APPLICATION_CREDENTIALS environment variable')

## 3) Authenticate and basic GCS example

This cell shows how to upload/download files to Google Cloud Storage. In Kaggle, ensure the service account has Storage permissions.


In [ ]:
try:
    from google.cloud import storage
    client = storage.Client(project=PROJECT_ID)
    bucket = client.bucket(BUCKET)
    print('GCS client ready for project', PROJECT_ID)
except Exception as e:
    print('GCS client not available or credentials missing:', e)

# Example functions
def upload_to_gcs(local_path, bucket_name, gcs_path):
    client = storage.Client(project=PROJECT_ID)
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(gcs_path)
    blob.upload_from_filename(local_path)
    print(f'Uploaded {local_path} -> gs://{bucket_name}/{gcs_path}')

def download_from_gcs(bucket_name, gcs_path, local_path):
    client = storage.Client(project=PROJECT_ID)
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(gcs_path)
    blob.download_to_filename(local_path)
    print(f'Downloaded gs://{bucket_name}/{gcs_path} -> {local_path}')

# Sanity check (list first 10 objects) - requires permission
try:
    blobs = list(bucket.list_blobs(max_results=10))
    print('First 10 objects in bucket:')
    for b in blobs:
        print('-', b.name)
except Exception as e:
    print('Could not list bucket contents (check permissions):', e)

## 4) BigQuery: load CSV and query (EMR / Capstone data)

Use BigQuery for tabular EMR exports or large datasets. This example loads a local CSV into BigQuery and runs a simple query. Ensure the service account has BigQuery Data Editor or Owner role for load operations.


In [ ]:
try:
    from google.cloud import bigquery
    bq = bigquery.Client(project=PROJECT_ID)
    print('BigQuery client ready')
except Exception as e:
    print('BigQuery client not available or credentials missing:', e)

# Example: load CSV to BigQuery (uncomment and edit paths to run)
# dataset_id = f"{PROJECT_ID}.capstone_demo"
# table_id = f"{dataset_id}.emr_records"
# bq.create_dataset(dataset_id, exists_ok=True)
# job_config = bigquery.LoadJobConfig(autodetect=True, write_disposition='WRITE_TRUNCATE')
# with open('/kaggle/input/my-emr/emr_export.csv','rb') as f:
#     load_job = bq.load_table_from_file(f, table_id, job_config=job_config)
# load_job.result()
# print('Loaded EMR CSV to', table_id)

# Example query (replace table_id)
# sql = f'SELECT * FROM `{table_id}` LIMIT 10'
# df = bq.query(sql).to_dataframe()
# df.head()

## 5) Cloud SQL: connect (optional)

If you have an operational Cloud SQL instance, use the Cloud SQL Python connector or SQLAlchemy. For Kaggle, use private IP or configure an authorized network (be cautious). Alternatively, use exported CSVs or BigQuery for analysis in Kaggle.


In [ ]:
# Example: SQLAlchemy connection (do not hardcode credentials)
# from sqlalchemy import create_engine
# DB_USER = 'db_user'
# DB_PASS = os.environ.get('CLOUD_SQL_DB_PASS')
# DB_NAME = 'clinicaldb'
# DB_HOST = '34.123.45.67:5432'
# engine = create_engine(f'postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}')
# with engine.connect() as conn:
#     print(conn.execute('SELECT 1').fetchall())
pass

## 6) Vertex AI: embeddings & LLM pattern (safe)

Kaggle notebooks can call Vertex AI endpoints. Prefer de-identified text and log requests. The Vertex SDK evolves; below is a pattern (pseudo-code) — replace with the appropriate `google-cloud-aiplatform` calls for your SDK version.


In [ ]:
# PSEUDO-CODE: Replace with your Vertex SDK calls
# from google.cloud import aiplatform
# aiplatform.init(project=PROJECT_ID, location=REGION)
# def get_embeddings(texts, model='textembedding-gecko@001'):
#     model = aiplatform.TextEmbeddingModel.from_pretrained(model)
#     embs = model.get_embeddings(texts)
#     return embs
#
# def call_vertex_llm(prompt, model='text-bison@001'):
#     model = aiplatform.TextGenerationModel.from_pretrained(model)
#     out = model.predict(prompt)
#     return out.text
print('Vertex AI pattern cell - implement with your SDK version')

## 7) RAG demo: toy data + TF retriever (works offline in Kaggle)

This is the same lightweight retriever used in the project: term-frequency vectors + cosine similarity. It runs without external libraries and demonstrates the end-to-end flow.


In [ ]:
# Toy docs
toy_docs = [
    {'id':'doc1','text':'68-year-old male with chest pain, troponin elevated, ST-elevation.'},
    {'id':'doc2','text':'45F with fever, productive cough, right lower lobe consolidation - pneumonia.'},
    {'id':'doc3','text':'70F with dyspnea, elevated BNP, reduced EF consistent with heart failure.'}
]

# Build TF index
import re, math
vocab = {}
doc_freqs = {}
for doc in toy_docs:
    tokens = re.findall(r"[a-z0-9]+", doc['text'].lower())
    freqs = {}
    for t in tokens:
        freqs[t] = freqs.get(t,0)+1
        if t not in vocab:
            vocab[t] = len(vocab)
    doc_freqs[doc['id']] = freqs
vectors = {}
for did, freqs in doc_freqs.items():
    vec = [0]*len(vocab)
    for term,count in freqs.items():
        vec[vocab[term]] = count
    vectors[did] = vec

# Retriever
def cosine(a,b):
    dot = sum(x*y for x,y in zip(a,b))
    na = math.sqrt(sum(x*x for x in a))
    nb = math.sqrt(sum(x*x for x in b))
    if na==0 or nb==0: return 0.0
    return dot/(na*nb)

def query_vec(q):
    tokens = re.findall(r"[a-z0-9]+", q.lower())
    vec = [0]*len(vocab)
    for t in tokens:
        if t in vocab:
            vec[vocab[t]] += 1
    return vec

def retrieve(query, top_k=2):
    qv = query_vec(query)
    scores = []
    for did, dv in vectors.items():
        # pad
        if len(dv) < len(qv):
            dv = dv + [0]*(len(qv)-len(dv))
        elif len(dv) > len(qv):
            qv = qv + [0]*(len(dv)-len(qv))
        scores.append((did, cosine(qv,dv)))
    scores.sort(key=lambda x: x[1], reverse=True)
    return scores[:top_k]

print('Retrieve example:', retrieve('chest pain troponin ST-elevation'))

## 8) Mock LLM + orchestrator (offline demo)

A deterministic mock LLM concatenates evidence and emits a cautious summary. For production, replace with Vertex or other LLM calls and ensure de-identification.


In [ ]:
def mock_llm(retrieved_docs):
    out = ['EVIDENCE:']
    for d in retrieved_docs:
        out.append(f"- {d['id']}: {d['text']}")
    summary = 'Summary: ' + ('; '.join([d['text'] for d in retrieved_docs]) )
    out.append('\n' + summary)
    out.append('\nRecommendation: Validate findings with primary record; consult specialists.')
    return '\n'.join(out)

# Orchestrator
def run_query(query):
    hits = retrieve(query, top_k=2)
    retrieved = []
    for did, score in hits:
        text = next(d['text'] for d in toy_docs if d['id']==did)
        retrieved.append({'id':did,'score':score,'text':text})
    return mock_llm(retrieved)

print(run_query('fever cough consolidation'))

## 9) Kaggle Capstone tips
- Use BigQuery for large datasets and exploratory SQL queries. Export features/tables to CSV for model training in Kaggle.
- Mount GCS buckets for large artifact storage (models, indexes).
- For reproducibility, save index files and model artifacts to GCS with versioned paths.
- Use Kaggle's dataset publishing to share non-sensitive parts of your capstone (models, notebooks) and avoid PHI.



## 10) Safety, compliance & next steps
- This notebook is a demo. **Do not** use outputs for autonomous clinical decisions.
- De-identify PHI before sending data to external services.
- Use Secret Manager and least-privilege service accounts.
- For production: implement end-to-end tests, CI, audit logs (BigQuery), and governance reviews.



## Files and artifacts
This notebook assumes you'll attach a private Kaggle dataset containing your service account JSON (for GCP access). It also illustrates saving an index file to GCS if you choose to build embeddings/FAISS locally in Kaggle.



----

If you'd like, I can now:
- Save this notebook to `/mnt/data/kaggle_capstone_notebook.ipynb` for download (I will do that now).
- Add cells that build an embedding-based FAISS index in Kaggle (requires internet & optional installs).
- Create a companion `README.md` for Kaggle dataset publishing tips.

Tell me which you prefer next.